In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

tf.__version__

'2.0.0'

# 당뇨병을 진단해봅시다.

In [3]:
xy = np.loadtxt('../Day_1/data-03-diabetes.csv', delimiter=',', dtype=np.float32)
x_train = xy[0:-100, 0:-1]
y_train = xy[0:-100, [-1]]

x_test = xy[-100:, 0:-1]
y_test = xy[-100:, [-1]]

print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

(659, 8) (659, 1)
(100, 8) (100, 1)


In [4]:
print(x_test)
print(y_test)

[[-0.294118   -0.0150754  -0.0491803  -0.333333   -0.550827    0.0134128
  -0.699402   -0.266667  ]
 [ 0.0588235   0.547739    0.278689   -0.393939   -0.763593   -0.0789866
  -0.926558   -0.2       ]
 [-0.294118    0.658291    0.114754   -0.474747   -0.602837    0.00149028
  -0.527754   -0.0666667 ]
 [-0.882353   -0.00502513 -0.0491803  -0.79798     0.         -0.242921
  -0.596072    0.        ]
 [ 0.176471   -0.316583    0.737705   -0.535354   -0.884161    0.0581222
  -0.823228   -0.133333  ]
 [-0.647059    0.236181    0.639344   -0.292929   -0.432624    0.707899
  -0.315115   -0.966667  ]
 [-0.0588235  -0.0854271   0.344262    0.          0.          0.0611028
  -0.565329    0.566667  ]
 [-0.294118    0.959799    0.147541    0.          0.         -0.0789866
  -0.786507   -0.666667  ]
 [ 0.0588235   0.567839    0.409836    0.          0.         -0.260805
  -0.870196    0.0666667 ]
 [ 0.         -0.0653266  -0.0163934   0.          0.          0.052161
  -0.842015   -0.866667  ]
 [-

In [5]:
dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(len(x_train))


In [7]:
W = tf.Variable(tf.random.normal([8, 1]), name='weight')
b = tf.Variable(tf.random.normal([1]), name='bias')


## 가설 설정
* 병이 있다 / 없다로 분류
* softmax classification으로 진행

## $$ y_k = \frac{exp(x_k)}{\sum_{i=1}^{n}exp(x_i)}  $$

In [8]:
def logistic_regression(features):
    hypothesis  = tf.div(1., 1. + tf.exp(-(tf.matmul(features, W) + b)))
    return hypothesis
  
print(logistic_regression(x_train))

AttributeError: module 'tensorflow' has no attribute 'div'

## Loss Function

* 기존 MSE 대신 Cross Entropy 사용

## $$
\begin{align}
loss(h(x),y) & = −y log(h(x))−(1−y)log(1−h(x))
\end{align}
$$

In [ ]:
def loss_fn(hypothesis, labels):
    loss = -tf.reduce_mean(labels * tf.log(hypothesis) + \
                           (1 - labels) * tf.log(1 - hypothesis))
    return loss

optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.005)

## 학습

In [ ]:
epochs = 5000

for step in range(epochs):
  for features, labels in dataset:
    with tf.GradientTape() as tape:
      loss_value = loss_fn(logistic_regression(features),labels)
      grads = tape.gradient(loss_value, [W,b])
      optimizer.apply_gradients(grads_and_vars=zip(grads,[W,b]))
      if step % 100 == 0:
            print("Iter: {}, Loss: {:.4f}".format(step, loss_fn(logistic_regression(features),labels)))

## 테스트

In [ ]:
def accuracy_fn(hypothesis, labels):
    predicted = tf.cast(hypothesis > 0.5, dtype=tf.int32)
    accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, labels), dtype=tf.float32))
    return accuracy

In [ ]:
test_acc = accuracy_fn(logistic_regression(x_test),y_test)
print("Testset Accuracy: {:.4f}".format(test_acc))